In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.float_format', '{:.2f}'.format)

## MultiIndexing in Series and DataFrame

In [2]:
sales_data = [100, 90, 105, 115, 85, 88, 95, 80, 110, 120, 100, 105]

index_tuples = [
    ('Laptops', 2020, 'USD'),
    ('Smartphones', 2020, 'EUR'),
    ('Headphones', 2020, 'GBP'),
    ('Laptops', 2021, 'USD'),
    ('Smartphones', 2021, 'EUR'),
    ('Headphones', 2021, 'GBP'),
    ('Laptops', 2022, 'USD'),
    ('Smartphones', 2022, 'EUR'),
    ('Headphones', 2022, 'JPY'),
    ('Laptops', 2023, 'USD'),
    ('Smartphones', 2023, 'JPY'),
    ('Headphones', 2023, 'GBP')
]

In [3]:
series = pd.Series(sales_data, index=index_tuples)
series

(Laptops, 2020, USD)        100
(Smartphones, 2020, EUR)     90
(Headphones, 2020, GBP)     105
(Laptops, 2021, USD)        115
(Smartphones, 2021, EUR)     85
(Headphones, 2021, GBP)      88
(Laptops, 2022, USD)         95
(Smartphones, 2022, EUR)     80
(Headphones, 2022, JPY)     110
(Laptops, 2023, USD)        120
(Smartphones, 2023, JPY)    100
(Headphones, 2023, GBP)     105
dtype: int64

In [5]:
row_data = series[('Laptops', 2020, 'USD')]
row_data

np.int64(100)

• Tuple-Based Indexing: The primary issue with this approach is that the
entire tuple must be known and used to fetch data. If you only have partial
information (e.g., you only know the fruit and year but not the currency), you’d
have difficulty accessing the desired data directly.

• Error-Prone: As in the provided code, if you try to fetch data with a tuple that
doesn’t exist in the index (like (Laptops, 2020, NY)), it will result in a KeyError.
This makes the process error-prone and requires additional code to handle
exceptions.

• Inefficient Filtering: If you want to filter data based on a specific element of
the tuple (e.g., all sales from 2021), you’ll have to write custom filtering logic,
making the process slower and less intuitive.

### Solution: Hierarchical Indexing (MultiIndex)

**MultiIndex.from_tuples**

In [6]:
multiindex = pd.MultiIndex.from_tuples(index_tuples, names=['Product', 'Year', 'Currency'])

In [7]:
series = pd.Series(sales_data, index=multiindex)
series

Product      Year  Currency
Laptops      2020  USD         100
Smartphones  2020  EUR          90
Headphones   2020  GBP         105
Laptops      2021  USD         115
Smartphones  2021  EUR          85
Headphones   2021  GBP          88
Laptops      2022  USD          95
Smartphones  2022  EUR          80
Headphones   2022  JPY         110
Laptops      2023  USD         120
Smartphones  2023  JPY         100
Headphones   2023  GBP         105
dtype: int64

In [8]:
data_2020 = series[:, 2020, :]
data_2020

Product      Currency
Laptops      USD         100
Smartphones  EUR          90
Headphones   GBP         105
dtype: int64

In [9]:
smartphones_sales = series['Smartphones']
smartphones_sales

Year  Currency
2020  EUR          90
2021  EUR          85
2022  EUR          80
2023  JPY         100
dtype: int64

**MultiIndex.from_product**

In [10]:
sales_data = [100, 90, 105, 115]

products = ['Smartphones', 'Laptops']
years = [2020, 2021]
currencies = ['USD', 'EUR']

In [11]:
index = pd.MultiIndex.from_product([products, years, currencies], names=['Product', 'Year', 'Currency'])

In [13]:
series = pd.Series(sales_data * 2, index=index)
series

Product      Year  Currency
Smartphones  2020  USD         100
                   EUR          90
             2021  USD         105
                   EUR         115
Laptops      2020  USD         100
                   EUR          90
             2021  USD         105
                   EUR         115
dtype: int64

In [14]:
smartphones_sales = series['Smartphones']
smartphones_sales

Year  Currency
2020  USD         100
      EUR          90
2021  USD         105
      EUR         115
dtype: int64

In [2]:
data = [
    [100, 101, 102, 103],
    [90, 91, 92, 93],
    [105, 106, 107, 108],
    [115, 116, 117, 118]
]

In [3]:
products = ['Smartphones', 'Laptops']
years = [2020, 2021]
currencies = ['USD', 'EUR', 'GBP', 'JPY']

In [4]:
row_index = pd.MultiIndex.from_product([products, years], names=['Product', 'Year'])

df = pd.DataFrame(data, index=row_index, columns=currencies)
df

USD  EUR  GBP  JPY
Product     Year                    
Smartphones 2020  100  101  102  103
            2021   90   91   92   93
Laptops     2020  105  106  107  108
            2021  115  116  117  118

In [19]:
df.loc['Smartphones']

,USD,EUR,GBP,JPY
Year,,,,
2020,100,101,102,103
2021,90,91,92,93


In [20]:
df.loc[('Smartphones', 2020)]

USD    100
EUR    101
GBP    102
JPY    103
Name: (Smartphones, 2020), dtype: int64

In [21]:
df.loc[('Smartphones', 2020), 'USD']

np.int64(100)

Slicing:

You can slice a multi-index DataFrame as well. To get data for smartphones for the
years 2020 and 2021:

In [22]:
df.loc[('Smartphones', slice(None)), :]

USD  EUR  GBP  JPY
Product     Year                    
Smartphones 2020  100  101  102  103
            2021   90   91   92   93

In [5]:
df.loc[('Smartphones', slice(None)), 'USD':'GBP']

USD  EUR  GBP
Product     Year               
Smartphones 2020  100  101  102
            2021   90   91   92

In [14]:
df.loc[('Smartphones', 2020:2021), 'USD':'GBP']

SyntaxError: invalid syntax (2290689352.py, line 1)

In [11]:
idx = pd.IndexSlice
df.loc[idx['Smartphones', 2020:2021], 'USD':'GBP']

UnsortedIndexError: 'MultiIndex slicing requires the index to be lexsorted: slicing on levels [1], lexsort depth 0'

In [12]:
df.index.is_monotonic_increasing

False

In [13]:
df = df.sort_index()

idx = pd.IndexSlice
df.loc[idx['Smartphones', 2020:2021], 'USD':'GBP']

USD  EUR  GBP
Product     Year               
Smartphones 2020  100  101  102
            2021   90   91   92

In [15]:
df['USD']

Product      Year
Laptops      2020    105
             2021    115
Smartphones  2020    100
             2021     90
Name: USD, dtype: int64

## Long and Wide Formats

### Long Format

• **Multiple Rows per Entity**: Long format data often has multiple rows for each
entity (e.g., individual, time point, location). These rows contain observations
related to the entity.

• **Variable Names in a Single Column**: Instead of having different columns for
each variable, the variable names are typically stored in a single column, often
referred to as the “variable” or “feature” column.

• **Values Correspond to Observations**: The actual values of the variables are
stored in a separate column, often referred to as the “value” column.

The long format is particularly useful for:

• Data Analysis: It facilitates various data analysis tasks such as aggregation,
filtering, and grouping.

• Visualization: Many data visualization tools and libraries, including Seaborn
and Plotly, work seamlessly with data in this format.

• Time Series Data: The long format is well-suited for time series data with
multiple time points for each entity.

In [16]:
data = {
    'Student': ['Alice', 'Bob', 'Alice', 'Bob'],
    'Subject': ['Math', 'Math', 'Science', 'Science'],
    'Score': [90, 88, 85, 92]
}

df_long = pd.DataFrame(data)

print('Long-format DataFrame:')
df_long

Long-format DataFrame:


,Student,Subject,Score
0,Alice,Math,90
1,Bob,Math,88
2,Alice,Science,85
3,Bob,Science,92


### Wide Format

• **Single Row per Entity**: Each entity has a single row, making it easy to compare
variables across entities.

• **Columns Represent Variables**: Variables are stored as columns, and each
column header indicates the variable’s name or category.

• **Values Correspond to Variables**: The values in each cell correspond to the
observations for a specific variable.

The wide format is well-suited for:

• Data Visualization: It is often more intuitive to visualize data using tools like
Excel or Tableau, which expect data in this format.

• Modeling: Some machine learning algorithms and statistical models may
require data in a wide format.

### Choosing the Right Format

• Use the long format for most data analysis tasks, as it provides flexibility and
compatibility with various analysis and visualization tools.

• Consider converting data into a wide format when you need it for specific
modeling purposes or when creating summary tables for reporting and
visualization.

In [17]:
data_wide = {
    'Student': ['Alice', 'Bob'],
    'Math': [90, 88],
    'Science': [85, 92]
}
df_wide = pd.DataFrame(data_wide)
print('Wide-format DataFrame:')
df_wide

Wide-format DataFrame:


,Student,Math,Science
0,Alice,90,85
1,Bob,88,92


## Stacking and Unstacking

In [18]:
data_wide = {
    'Quarter': ['Q1', 'Q2'],
    'Product_A': [1000, 1100],
    'Product_B': [800, 820],
    'Product_C': [1200, 1300]
}

df_wide = pd.DataFrame(data_wide)
df_wide

,Quarter,Product_A,Product_B,Product_C
0,Q1,1000,800,1200
1,Q2,1100,820,1300


In [19]:
# Stacking (vertical transformation) the data
stacked_df = df_wide.set_index('Quarter').stack().reset_index(name='Sales')
stacked_df

,Quarter,level_1,Sales
0,Q1,Product_A,1000
1,Q1,Product_B,800
2,Q1,Product_C,1200
3,Q2,Product_A,1100
4,Q2,Product_B,820
5,Q2,Product_C,1300


In [24]:
stacked_df.index

RangeIndex(start=0, stop=6, step=1)

In [25]:
stacked_df.columns

Index(['Quarter', 'level_1', 'Sales'], dtype='object')

In [22]:
print(df_wide.set_index('Quarter'))

         Product_A  Product_B  Product_C
Quarter                                 
Q1            1000        800       1200
Q2            1100        820       1300


In [23]:
print(df_wide.set_index('Quarter').stack())

Quarter           
Q1       Product_A    1000
         Product_B     800
         Product_C    1200
Q2       Product_A    1100
         Product_B     820
         Product_C    1300
dtype: int64


In [26]:
df_wide.set_index('Quarter').stack().index

MultiIndex([('Q1', 'Product_A'),
            ('Q1', 'Product_B'),
            ('Q1', 'Product_C'),
            ('Q2', 'Product_A'),
            ('Q2', 'Product_B'),
            ('Q2', 'Product_C')],
           names=['Quarter', None])

Upon applying the stack function with set_index, the result is a vertically transformed
DataFrame, often referred to as a “stacked” DataFrame. In the stacked DataFrame:

• The ‘Quarter’ column is retained as an identifier variable, and the DataFrame
is indexed by ‘Quarter.’

• The ‘stack’ function transforms the columns (Product_A, Product_B,
Product_C) into rows, effectively creating a multi-level index where ‘Quarter’
and ‘Product’ become the levels.

• The ‘reset_index’ function is used to reset the index and create a new column
named ‘Sales’ to store the sales figures.

In [27]:
data_long = {
    'Month': ['January', 'January', 'February', 'February'],
    'City': ['New York', 'Boston', 'New York', 'Boston'],
    'Temperature': [32, 30, 35, 32]
}

df_long = pd.DataFrame(data_long)
df_long

,Month,City,Temperature
0,January,New York,32
1,January,Boston,30
2,February,New York,35
3,February,Boston,32


In [28]:
# Unstacking (horizontal transformation) the data
unstacked_df = df_long.set_index(['Month', 'City'])['Temperature'].unstack().reset_index()
unstacked_df

City,Month,Boston,New York
0,February,32,35
1,January,30,32


## Melting Data

**The Melt Function in Pandas**

In Pandas, the melt function is used to melt data efficiently. Let’s explore the key
parameters of the melt function:

• **id_vars**: This parameter specifies which columns should remain as-is
(unmelted) and not be transformed into rows.

• **value_vars**: Here, you specify which columns should be melted into rows. If
not specified, all columns not mentioned in id_vars are melted.

• **var_name**: This parameter allows you to specify the name of the column that
will store the variable names after melting.

• **value_name**: You use this parameter to name the column that will store the
values after melting.

In [29]:
data_wide = {
    'Date': ['2023-01-01', '2023-01-02', '2023-01-03'],
    'Temperature': [32, 30, 35],
    'Humidity': [45, 48, 42]
}

df_wide = pd.DataFrame(data_wide)
df_wide

,Date,Temperature,Humidity
0,2023-01-01,32,45
1,2023-01-02,30,48
2,2023-01-03,35,42


In [30]:
# Melting the time series data
melted_df = df_wide.melt(id_vars=['Date'], var_name='Variable', value_name='Value')
melted_df

,Date,Variable,Value
0,2023-01-01,Temperature,32
1,2023-01-02,Temperature,30
2,2023-01-03,Temperature,35
3,2023-01-01,Humidity,45
4,2023-01-02,Humidity,48
5,2023-01-03,Humidity,42


## Exploding Data

In [31]:
data = {
    'Student ID': [1, 2, 3],
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Courses': [['Math', 'Science'], ['History', 'Geography'], ['English']]
}

df = pd.DataFrame(data)
df

,Student ID,Name,Courses
0,1,Alice,"[Math, Science]"
1,2,Bob,"[History, Geography]"
2,3,Charlie,[English]


In [32]:
# Using explode to expand the 'Courses' column
exploded_df = df.explode('Courses', ignore_index=True)
exploded_df

,Student ID,Name,Courses
0,1,Alice,Math
1,1,Alice,Science
2,2,Bob,History
3,2,Bob,Geography
4,3,Charlie,English


## Creating Pivot Tables

In [33]:
data = {
    'Product Category': ['Electronics', 'Clothing', 'Electronics', 'Clothing'],
    'Sales Region': ['North', 'South', 'North', 'South'],
    'Revenue': [5000, 3000, 6000, 4000]
}

df = pd.DataFrame(data)
df

,Product Category,Sales Region,Revenue
0,Electronics,North,5000
1,Clothing,South,3000
2,Electronics,North,6000
3,Clothing,South,4000


In [34]:
pivot_table = pd.pivot_table(df, 
                             values='Revenue', 
                             index='Product Category', 
                             columns='Sales Region', 
                             aggfunc='sum', 
                             fill_value=0)
pivot_table

Sales Region,North,South
Product Category,,
Clothing,0,7000
Electronics,11000,0


## Advanced Pivot Table Techniques

**Multi-Index Pivot Tables**

In [39]:
data = {
    'Year': [2021, 2021, 2022, 2022],
    'Quarter': ['Q1', 'Q2', 'Q1', 'Q2'],
    'Product Category': ['Electronics', 'Clothing', 'Electronics', 'Clothing'],
    'Region': ['North', 'South', 'North', 'South'],
    'Revenue': [5000, 3000, 6000, 4000]
}

df = pd.DataFrame(data)
df

,Year,Quarter,Product Category,Region,Revenue
0,2021,Q1,Electronics,North,5000
1,2021,Q2,Clothing,South,3000
2,2022,Q1,Electronics,North,6000
3,2022,Q2,Clothing,South,4000


In [40]:
pivot_table = df.pivot_table(index=['Year', 'Quarter'],
                             columns=['Product Category', 'Region'],
                             values='Revenue',
                             aggfunc='sum',
                             fill_value=0)
pivot_table

Product Category Clothing Electronics
Region              South       North
Year Quarter                         
2021 Q1                 0        5000
     Q2              3000           0
2022 Q1                 0        6000
     Q2              4000           0

In [41]:
pivot_table = df.pivot_table(index='Year', 
                             columns=['Product Category', 'Region'], 
                             values='Revenue', 
                             aggfunc='sum', 
                             fill_value=0)
pivot_table

Product Category,Clothing,Electronics
Region,South,North
Year,,
2021,3000,5000
2022,4000,6000
